In [19]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from sklearn.model_selection import KFold

In [20]:
train_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/train.csv')
test_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/test.csv')

y_train = train_df["author"].values

In [21]:
with open('keras_input_train.pkl', 'rb') as f:
    sequences = pickle.load(f)
with open('keras_input_test.pkl', 'rb') as f:
    test_sequences = pickle.load(f)
    
x_train = sequences
x_test = test_sequences

print(x_train.shape)
print(x_test.shape)

(19579, 24992)
(8392, 24992)


In [22]:
%%time
# use cross validation

SPLIT = 10
train_pred = 0

print("The number of splitting iterations in the cross-validator : {}".format(SPLIT))
print()

gnb =  MultinomialNB()
kf = KFold(n_splits=SPLIT,shuffle=True)
for train_index, test_index in kf.split(x_train):
    gnb.fit(x_train[train_index].todense(), y_train[train_index])
    
    y_pred = gnb.predict(x_train[test_index].todense())
    s = accuracy_score(y_train[test_index],y_pred)
    print("The accuracy score of cross validation is {}".format(s))
    
    train_pred += s/SPLIT

print()    
print("The mean of the cross validation score is {}".format(train_pred))

The number of splitting iterations in the cross-validator : 10

The accuracy score of cross validation is 0.8140960163432074
The accuracy score of cross validation is 0.8166496424923391
The accuracy score of cross validation is 0.8100102145045965
The accuracy score of cross validation is 0.8350357507660878
The accuracy score of cross validation is 0.8038815117466803
The accuracy score of cross validation is 0.8217568947906027
The accuracy score of cross validation is 0.8084780388151175
The accuracy score of cross validation is 0.8391215526046987
The accuracy score of cross validation is 0.8381001021450459
The accuracy score of cross validation is 0.8247317322432295

The mean of the cross validation score is 0.8211861456451606
Wall time: 37.3 s


In [23]:
y_pred = gnb.predict(x_train.todense())
s = accuracy_score(y_train,y_pred)
print("The accuracy score {}".format(s))

The accuracy score 0.9032126257725114


In [24]:
print(gnb.classes_)
predict_probability = gnb.predict_proba(x_test)
print(predict_probability.shape)

['EAP' 'HPL' 'MWS']
(8392, 3)


In [25]:
# submission

submission = pd.DataFrame(predict_probability, columns=gnb.classes_)
submission["id"] = test_df["id"]
cols = submission.columns.tolist()
cols = cols[-1:]+cols[:-1]
submission = submission[cols]
submission.head(10)

,id,EAP,HPL,MWS
0,id02310,0.254412,0.107675,0.637912
1,id24541,0.756738,0.143498,0.099764
2,id00134,0.513215,0.361134,0.125651
3,id27757,0.598115,0.342112,0.059773
4,id04081,0.704614,0.168976,0.126411
5,id27337,0.580908,0.293597,0.125495
6,id24265,0.530229,0.309271,0.160499
7,id25917,0.163623,0.222553,0.613824
8,id04951,0.835742,0.119795,0.044463
9,id14549,0.642689,0.164173,0.193138


In [26]:
submission.to_csv('submission_NB.csv',index=False)